Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
import keras
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Lungs_CT_Covid19_Detect/data_set/training_labels.csv')
Y = train_data['lable']
kf = KFold(n_splits = 5)

In [ ]:
idg = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.05,
    rescale=1./255,) 

In [ ]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

## Pre-trained VGG16

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
VALIDATION_RECALL=[]
VALIDATION_PRECISION=[]
VALIDATION_AUC=[]
opt = SGD(lr=1e-5)
n=746
image_dir='/content/drive/MyDrive/Lungs_CT_Covid19_Detect/data_set'
save_dir = '/content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/'
fold_var = 1
for train_index, val_index in kf.split(np.zeros(n),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    
    train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
                    x_col = "filename", y_col = "lable",target_size=(248,248),
                    class_mode = "raw",batch_size=64, shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = image_dir,
                x_col = "filename", y_col = "lable",target_size=(248,248),
                class_mode = "raw", batch_size=64,shuffle = True)
    #create model
    base_model = VGG16(input_shape = (248, 248, 3), 
    include_top = False, 
    weights = 'imagenet',
    pooling='max')
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    # add output layer 
    out_layer = Dense(1, activation='sigmoid')(base_model.layers[-1].output)
    model_pretrained_vgg = Model(base_model.input, out_layer)
    # compile model
    model_pretrained_vgg.compile(optimizer=opt,
                      loss='binary_crossentropy',
                      metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),tf.keras.metrics.AUC(
            num_thresholds=200, curve='ROC',
            summation_method='interpolation', name=None, dtype=None,
            thresholds=None, multi_label=False, label_weights=None
        ),'accuracy'])
  
    #create callbacks
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                  monitor='val_accuracy', verbose=1, 
                  save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # FIT THE MODEL
    history = model_pretrained_vgg.fit(train_data_generator,
              epochs=10,
              callbacks=callbacks_list,
              validation_data=valid_data_generator)
    model_pretrained_vgg.load_weights("/content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/model_"+str(fold_var)+".h5")
	
    results = model_pretrained_vgg.evaluate(valid_data_generator)
    results = dict(zip(model_pretrained_vgg.metrics_names,results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])

   
    
    
    
    tf.keras.backend.clear_session()
    
    fold_var += 1

Found 596 validated image filenames.
Found 150 validated image filenames.
58892288/58889256 [==============================] - 0s 0us/step
Epoch 1/10
10/10 [==============================] - 289s 27s/step - loss: 0.9816 - recall: 0.9630 - precision: 0.3723 - auc: 0.4781 - accuracy: 0.3719 - val_loss: 0.3198 - val_recall: 0.9867 - val_precision: 1.0000 - val_auc: 0.0000e+00 - val_accuracy: 0.9867

Epoch 00001: val_accuracy improved from -inf to 0.98667, saving model to /content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/model_1.h5
Epoch 2/10
10/10 [==============================] - 15s 1s/step - loss: 1.0016 - recall: 0.9716 - precision: 0.3367 - auc: 0.5200 - accuracy: 0.3462 - val_loss: 0.3437 - val_recall: 0.9667 - val_precision: 1.0000 - val_auc: 0.0000e+00 - val_accuracy: 0.9667

Epoch 00002: val_accuracy did not improve from 0.98667
Epoch 3/10
10/10 [==============================] - 15s 1s/step - loss: 0.9821 - recall: 0.9822 - precision: 0.3269 - auc: 0.4913 - accuracy: 

In [ ]:
print(np.average(VALIDATION_ACCURACY))

0.44430425483733416


## Pre-trained MobileNet

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
VALIDATION_RECALL=[]
VALIDATION_PRECISION=[]
VALIDATION_AUC=[]

n=746
image_dir='/content/drive/MyDrive/Lungs_CT_Covid19_Detect/data_set'
save_dir = '/content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/'
fold_var = 1
for train_index, val_index in kf.split(np.zeros(n),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    
    train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
                    x_col = "filename", y_col = "lable",target_size=(248,248),
                    class_mode = "raw",batch_size=64, shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = image_dir,
                x_col = "filename", y_col = "lable",target_size=(248,248),
                class_mode = "raw", batch_size=64,shuffle = True)
    #create model
    from keras.applications import MobileNet
    MobileNet=MobileNet(input_shape = (128, 128, 3),weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    x=MobileNet.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    x=Dense(1024,activation='relu')(x) #dense layer 2
    x=Dense(512,activation='relu')(x) #dense layer 3
    preds=Dense(1,activation='sigmoid')(x) #final layer with sigmoid activation

    model_mobileNet_pretrained = tf.keras.models.Model(MobileNet.input, outputs=preds)
    # compile model
    model_mobileNet_pretrained.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),tf.keras.metrics.AUC(
          num_thresholds=200, curve='ROC',
          summation_method='interpolation', name=None, dtype=None,
          thresholds=None, multi_label=False, label_weights=None
      ),'accuracy'])
  
    #create callbacks
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                  monitor='val_accuracy', verbose=1, 
                  save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # FIT THE MODEL
    history = model_mobileNet_pretrained.fit(train_data_generator,
              epochs=10,
              callbacks=callbacks_list,
              validation_data=valid_data_generator)
    model_mobileNet_pretrained.load_weights("/content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/model_"+str(fold_var)+".h5")
	
    results = model_mobileNet_pretrained.evaluate(valid_data_generator)
    results = dict(zip(model_mobileNet_pretrained.metrics_names,results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])

   
    
    
    
    tf.keras.backend.clear_session()
    
    fold_var += 1

Found 596 validated image filenames.
Found 150 validated image filenames.
Epoch 1/10
10/10 [==============================] - 22s 2s/step - loss: 1.0287 - recall: 0.4606 - precision: 0.4767 - auc: 0.6195 - accuracy: 0.6335 - val_loss: 0.2581 - val_recall: 0.9400 - val_precision: 1.0000 - val_auc: 0.0000e+00 - val_accuracy: 0.9400

Epoch 00001: val_accuracy improved from -inf to 0.94000, saving model to /content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/model_1.h5
Epoch 2/10
10/10 [==============================] - 17s 2s/step - loss: 0.3346 - recall: 0.8243 - precision: 0.8006 - auc: 0.9306 - accuracy: 0.8683 - val_loss: 0.4769 - val_recall: 0.7867 - val_precision: 1.0000 - val_auc: 0.0000e+00 - val_accuracy: 0.7867

Epoch 00002: val_accuracy did not improve from 0.94000
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: 0.1977 - recall: 0.9353 - precision: 0.8461 - auc: 0.9785 - accuracy: 0.9224 - val_loss: 1.1439 - val_recall: 0.3933 - val_precision: 1.0

In [ ]:
print(np.average(VALIDATION_ACCURACY))

0.9060402631759643


## Pre-trained ResNet50

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
VALIDATION_RECALL=[]
VALIDATION_PRECISION=[]
VALIDATION_AUC=[]

n=746
image_dir='/content/drive/MyDrive/Lungs_CT_Covid19_Detect/data_set'
save_dir = '/content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/'
fold_var = 1
for train_index, val_index in kf.split(np.zeros(n),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    
    train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
                    x_col = "filename", y_col = "lable",target_size=(248,248),
                    class_mode = "raw",batch_size=64, shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = image_dir,
                x_col = "filename", y_col = "lable",target_size=(248,248),
                class_mode = "raw", batch_size=64,shuffle = True)
    #create model
    from keras.applications import ResNet50
    modelResNet = ResNet50()
    base_ResNet = ResNet50(input_shape=(128, 128,3), include_top=False, weights="imagenet")

    for layer in base_ResNet.layers:
            layer.trainable = False

    base_ResNet = keras.models.Sequential()
    base_ResNet.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
    base_ResNet.add(Dense(1, activation='sigmoid'))
    # compile model
    base_ResNet.compile(optimizer = tf.keras.optimizers.SGD(lr=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy', 'Recall', 'Precision'])

    
 
    
  
    #create callbacks
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                  monitor='val_accuracy', verbose=1, 
                  save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # FIT THE MODEL
    history = base_ResNet.fit(train_data_generator,
              epochs=10,
              callbacks=callbacks_list,
              validation_data=valid_data_generator)
    base_ResNet.load_weights("/content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/model_"+str(fold_var)+".h5")
	
    results = base_ResNet.evaluate(valid_data_generator)
    results = dict(zip(base_ResNet.metrics_names,results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])

   
    
    
    
    tf.keras.backend.clear_session()
    
    fold_var += 1

Found 596 validated image filenames.
Found 150 validated image filenames.
Epoch 1/10
10/10 [==============================] - 257s 22s/step - loss: 2.1763 - accuracy: 0.5234 - recall: 0.4612 - precision: 0.3504 - val_loss: 2.1648 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to /content/drive/MyDrive/Lungs_CT_Covid19_Detect/Saved_Models/model_1.h5
Epoch 2/10
10/10 [==============================] - 16s 2s/step - loss: 1.5854 - accuracy: 0.6546 - recall: 0.4354 - precision: 0.4130 - val_loss: 2.7268 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/10
10/10 [==============================] - 16s 2s/step - loss: 1.4096 - accuracy: 0.6327 - recall: 0.4759 - precision: 0.4743 - val_loss: 3.3858 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00003: val_acc

In [ ]:
print(np.average(VALIDATION_ACCURACY))

0.4442326635122299
